## A convolutional autoencoder

Interesting note:
This model performs better on representing the data than the MLP AE. 
It is less sensitive to overfitting, because of the Dropout layers. 

When comparing the mean and standard deviation of input data and reproduced data, 
this model comes much closer to the original than the MLP does.


The model looks like this:

```_______________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_1 (InputLayer)         (None, 32, 1)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 32, 18)            54        
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 16, 18)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 16, 8)             296       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 8, 8)              0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 8, 8)              136       
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 4, 8)              0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 4, 8)              136       
_________________________________________________________________
up_sampling1d_1 (UpSampling1 (None, 8, 8)              0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 8, 8)              136       
_________________________________________________________________
up_sampling1d_2 (UpSampling1 (None, 16, 8)             0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 16, 18)            306       
_________________________________________________________________
up_sampling1d_3 (UpSampling1 (None, 32, 18)            0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 32, 1)             37        
=================================================================
Total params: 1,101
Trainable params: 1,101
Non-trainable params: 0```

In [1]:
# imports

from keras.layers import Input, Dense, Conv1D, MaxPooling1D, UpSampling1D
from keras.models import Model
import keras.backend as K
from keras.callbacks import EarlyStopping
import time

import numpy as np
from music21 import midi, stream

# Importing functions used in AE Midi
from create_m21_object import createMusic21Object
from create_pattern import createPattern

from sklearn.preprocessing import MinMaxScaler

# For training
epochs = 400
batch_size = 64

/home/vik/miniconda3/envs/magenta3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Get data
data = np.load("music.npz")
x_train = data['x_train']
x_test = data['x_test']

# Reshaping to use in a Conv1D network
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

x_train.shape, x_test.shape

((12475, 32, 1), (6237, 32, 1))

In [3]:
input_song = Input(shape=(32,1)) # channels last

### Encoder model

In the encoder model, each layer halfens the size of the input, ending up with a bottleneck of size 4.

In [4]:
# create encoder model

x = Conv1D(filters=18, kernel_size=(2), activation='relu', padding='same')(input_song)
x = MaxPooling1D((2), padding='same')(x)
x = Conv1D(filters=8, kernel_size=(2), activation='relu', padding='same')(x)
x = MaxPooling1D((2), padding='same')(x)
x = Conv1D(8, (2), activation='relu', padding='same')(x)
encoded = MaxPooling1D((2), padding='same')(x)

### Decoder model

In the decoder model, each layer doubles the size of its input, ending up with the output size equal to the input size.

In [5]:
# create decoder model

x = Conv1D(8, (2), activation='relu', padding='same')(encoded)
x = UpSampling1D((2))(x)
x = Conv1D(8, (2), activation='relu', padding='same')(x)
x = UpSampling1D((2))(x)
x = Conv1D(18, kernel_size=(2), activation='relu', padding='same')(x)
x = UpSampling1D((2))(x)
decoded = Conv1D(1, (2), activation='sigmoid', padding='same')(x)

In [6]:
autoencoder = Model(input_song, decoded)
autoencoder.compile(optimizer='rmsprop', loss='mean_squared_error')

earlystop = EarlyStopping(monitor='val_loss', min_delta=10e-5, patience=5,
                          verbose=1, mode='auto')

callbacks_list = [earlystop]

# train the model
start = time.time()

autoencoder.fit(x_train, x_train,
               epochs=epochs,
               batch_size=batch_size,
               shuffle=True,
               validation_split=0.2)

end = time.time()

print("time to train", end-start)

Train on 9980 samples, validate on 2495 samples
Epoch 1/400
9980/9980 [==============================] - 1s 138us/step - loss: 0.0515 - val_loss: 0.0383
Epoch 2/400
9980/9980 [==============================] - 1s 97us/step - loss: 0.0321 - val_loss: 0.0323
Epoch 3/400
9980/9980 [==============================] - 1s 95us/step - loss: 0.0301 - val_loss: 0.0313
Epoch 4/400
9980/9980 [==============================] - 1s 93us/step - loss: 0.0294 - val_loss: 0.0307
Epoch 5/400
9980/9980 [==============================] - 1s 93us/step - loss: 0.0290 - val_loss: 0.0307
Epoch 6/400
9980/9980 [==============================] - 1s 92us/step - loss: 0.0286 - val_loss: 0.0304
Epoch 7/400
9980/9980 [==============================] - 1s 94us/step - loss: 0.0284 - val_loss: 0.0301
Epoch 8/400
9980/9980 [==============================] - 1s 94us/step - loss: 0.0281 - val_loss: 0.0296
Epoch 9/400
9980/9980 [==============================] - 1s 93us/step - loss: 0.0279 - val_loss: 0.0293
Epoch 10/400
99

9980/9980 [==============================] - 1s 106us/step - loss: 0.0214 - val_loss: 0.0229
Epoch 79/400
9980/9980 [==============================] - 1s 95us/step - loss: 0.0214 - val_loss: 0.0230
Epoch 80/400
9980/9980 [==============================] - 1s 104us/step - loss: 0.0214 - val_loss: 0.0228
Epoch 81/400
9980/9980 [==============================] - 1s 100us/step - loss: 0.0214 - val_loss: 0.0229
Epoch 82/400
9980/9980 [==============================] - 1s 101us/step - loss: 0.0213 - val_loss: 0.0228
Epoch 83/400
9980/9980 [==============================] - 1s 110us/step - loss: 0.0213 - val_loss: 0.0230
Epoch 84/400
9980/9980 [==============================] - 1s 88us/step - loss: 0.0213 - val_loss: 0.0228
Epoch 85/400
9980/9980 [==============================] - 1s 86us/step - loss: 0.0212 - val_loss: 0.0228
Epoch 86/400
9980/9980 [==============================] - 1s 87us/step - loss: 0.0212 - val_loss: 0.0229
Epoch 87/400
9980/9980 [==============================] - 1s 87

9980/9980 [==============================] - 1s 81us/step - loss: 0.0190 - val_loss: 0.0211
Epoch 156/400
9980/9980 [==============================] - 1s 82us/step - loss: 0.0190 - val_loss: 0.0204
Epoch 157/400
9980/9980 [==============================] - 1s 83us/step - loss: 0.0190 - val_loss: 0.0205
Epoch 158/400
9980/9980 [==============================] - 1s 83us/step - loss: 0.0190 - val_loss: 0.0211
Epoch 159/400
9980/9980 [==============================] - 1s 82us/step - loss: 0.0189 - val_loss: 0.0203
Epoch 160/400
9980/9980 [==============================] - 1s 82us/step - loss: 0.0189 - val_loss: 0.0206
Epoch 161/400
9980/9980 [==============================] - 1s 83us/step - loss: 0.0189 - val_loss: 0.0203
Epoch 162/400
9980/9980 [==============================] - 1s 81us/step - loss: 0.0189 - val_loss: 0.0204
Epoch 163/400
9980/9980 [==============================] - 1s 82us/step - loss: 0.0189 - val_loss: 0.0202
Epoch 164/400
9980/9980 [==============================] - 1

9980/9980 [==============================] - 1s 89us/step - loss: 0.0179 - val_loss: 0.0193
Epoch 233/400
9980/9980 [==============================] - 1s 89us/step - loss: 0.0179 - val_loss: 0.0191
Epoch 234/400
9980/9980 [==============================] - 1s 103us/step - loss: 0.0179 - val_loss: 0.0191
Epoch 235/400
9980/9980 [==============================] - 1s 90us/step - loss: 0.0179 - val_loss: 0.0191
Epoch 236/400
9980/9980 [==============================] - 1s 87us/step - loss: 0.0179 - val_loss: 0.0192
Epoch 237/400
9980/9980 [==============================] - 1s 89us/step - loss: 0.0179 - val_loss: 0.0190
Epoch 238/400
9980/9980 [==============================] - 1s 90us/step - loss: 0.0179 - val_loss: 0.0196
Epoch 239/400
9980/9980 [==============================] - 1s 89us/step - loss: 0.0179 - val_loss: 0.0192
Epoch 240/400
9980/9980 [==============================] - 1s 85us/step - loss: 0.0178 - val_loss: 0.0191
Epoch 241/400
9980/9980 [==============================] - 

9980/9980 [==============================] - 1s 82us/step - loss: 0.0171 - val_loss: 0.0185
Epoch 310/400
9980/9980 [==============================] - 1s 85us/step - loss: 0.0171 - val_loss: 0.0184
Epoch 311/400
9980/9980 [==============================] - 1s 86us/step - loss: 0.0171 - val_loss: 0.0185
Epoch 312/400
9980/9980 [==============================] - 1s 82us/step - loss: 0.0171 - val_loss: 0.0183
Epoch 313/400
9980/9980 [==============================] - 1s 83us/step - loss: 0.0170 - val_loss: 0.0185
Epoch 314/400
9980/9980 [==============================] - 1s 82us/step - loss: 0.0170 - val_loss: 0.0181
Epoch 315/400
9980/9980 [==============================] - 1s 81us/step - loss: 0.0170 - val_loss: 0.0185
Epoch 316/400
9980/9980 [==============================] - 1s 82us/step - loss: 0.0170 - val_loss: 0.0188
Epoch 317/400
9980/9980 [==============================] - 1s 84us/step - loss: 0.0170 - val_loss: 0.0182
Epoch 318/400
9980/9980 [==============================] - 1

9980/9980 [==============================] - 1s 94us/step - loss: 0.0163 - val_loss: 0.0175
Epoch 387/400
9980/9980 [==============================] - 1s 94us/step - loss: 0.0163 - val_loss: 0.0174
Epoch 388/400
9980/9980 [==============================] - 1s 86us/step - loss: 0.0163 - val_loss: 0.0175
Epoch 389/400
9980/9980 [==============================] - 1s 86us/step - loss: 0.0162 - val_loss: 0.0178
Epoch 390/400
9980/9980 [==============================] - 1s 86us/step - loss: 0.0162 - val_loss: 0.0176
Epoch 391/400
9980/9980 [==============================] - 1s 87us/step - loss: 0.0163 - val_loss: 0.0175
Epoch 392/400
9980/9980 [==============================] - 1s 87us/step - loss: 0.0163 - val_loss: 0.0179
Epoch 393/400
9980/9980 [==============================] - 1s 100us/step - loss: 0.0162 - val_loss: 0.0175
Epoch 394/400
9980/9980 [==============================] - 1s 87us/step - loss: 0.0162 - val_loss: 0.0177
Epoch 395/400
9980/9980 [==============================] - 

In [7]:
# Importing feature range from MLP example
feature_range = np.load("feature_range.npy")

# Creating a scaler for the predicted values
predictScaler = MinMaxScaler(feature_range=feature_range)

song_prediction = autoencoder.predict(x_test)

# Reshape to allow scaler to work
song = song_prediction.reshape((song_prediction.shape[0], song_prediction.shape[1]))

song = (predictScaler.fit_transform(song)).astype('int')
newsong = createPattern(song[40])

play = createMusic21Object(newsong)

In [8]:
midi_stream = stream.Stream(play)

sp = midi.realtime.StreamPlayer(midi_stream)

sp.play()

In [9]:
song_prediction.mean(), x_test.mean(), song_prediction.std(), x_test.std()

(0.88665974, 0.8663733687342035, 0.15097632, 0.2015395137870709)

In [10]:
# autoencoder.summary()